In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chrisvarella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chrisvarella/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('covid19_dataset.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

40 documents
9 classes ['cleaning', 'goodbye', 'greeting', 'hygiene', 'isolate', 'masks', 'quarantine', 'spread', 'symptoms']
73 unique lemmatized words ['a', 'air', 'am', 'anyone', 'are', 'at', 'by', 'canada', 'clean', 'close', 'contact', 'corona', 'cough', 'covid-19', 'cya', 'day', 'do', 'doe', 'eye', 'face', 'from', 'good', 'goodbye', 'ha', 'had', 'hand', 'have', 'hello', 'hi', 'home', 'how', 'i', 'is', 'just', 'later', 'leaving', 'long', 'mask', 'mean', 'mouth', 'my', 'myself', 'non-medical', 'nose', 'of', 'often', 'outside', 'quarantine', 'returned', 'see', 'self-isolate', 'should', 'sneeze', 'someone', 'spread', 'stay', 'surface', 'symptom', 'the', 'there', 'to', 'touch', 'travel', 'up', 'virus', 'wash', 'wear', 'what', 'whats', 'when', 'who', 'with', 'you']


### Building the Deep Learning Model

In [6]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
40/40 [==============================] - 0s 7ms/step - loss: 2.2683 - accuracy: 0.0750
Epoch 2/200
40/40 [==============================] - 0s 661us/step - loss: 2.1481 - accuracy: 0.0750
Epoch 3/200
40/40 [==============================] - 0s 635us/step - loss: 2.0490 - accuracy: 0.2000
Epoch 4/200
40/40 [==============================] - 0s 707us/step - loss: 1.9166 - accuracy: 0.3500
Epoch 5/200
40/40 [==============================] - 0s 762us/step - loss: 1.8379 - accuracy: 0.3500
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 1.7957 - accuracy: 0.3750
Epoch 7/200
40/40 [==============================] - 0s 712us/step - loss: 1.6252 - accuracy: 0.4500
Epoch 8/200
40/40 [==============================] - 0s 706us/step - loss: 1.4966 - accuracy: 0.4500
Epoch 9/200
40/40 [==============================] - 0s 656us/step - loss: 1.3349 - accuracy: 0.6250
Epoch 10/200
40/40 [==============================] - 0s 702us/step - loss: 1.4172 - accuracy: 

40/40 [==============================] - 0s 696us/step - loss: 0.0229 - accuracy: 1.0000
Epoch 161/200
40/40 [==============================] - 0s 787us/step - loss: 0.0110 - accuracy: 1.0000
Epoch 162/200
40/40 [==============================] - 0s 546us/step - loss: 0.1118 - accuracy: 0.9750
Epoch 163/200
40/40 [==============================] - 0s 573us/step - loss: 0.0112 - accuracy: 1.0000
Epoch 164/200
40/40 [==============================] - 0s 614us/step - loss: 0.0204 - accuracy: 1.0000
Epoch 165/200
40/40 [==============================] - 0s 684us/step - loss: 0.0473 - accuracy: 1.0000
Epoch 166/200
40/40 [==============================] - 0s 592us/step - loss: 0.1043 - accuracy: 0.9500
Epoch 167/200
40/40 [==============================] - 0s 647us/step - loss: 0.0435 - accuracy: 0.9750
Epoch 168/200
40/40 [==============================] - 0s 631us/step - loss: 0.0504 - accuracy: 0.9750
Epoch 169/200
40/40 [==============================] - 0s 631us/step - loss: 0.0327 - a

In [8]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('covid19_dataset.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [10]:
res = chatbot_response("Should I self-isolate")
print(res)

You should self-isolate if diagnosed with COVID-19, if you have the symptoms of COVID-19, if you have been exposed to COVID-19 and/or returned from travel outside of Canada


In [11]:
strText = input("Please type a question related to COVID-19: ")
answer = chatbot_response(strText)
print("Here is your question about COVID-19: ", strText)
print("Here is the answer: ", answer)

Please type a question related to COVID-19: should I self-isolate
Here is your question about COVID-19:  should I self-isolate
Here is the answer:  You should self-isolate if diagnosed with COVID-19, if you have the symptoms of COVID-19, if you have been exposed to COVID-19 and/or returned from travel outside of Canada
